# OpenStreetMap Data Case Study



### Map Location

In this case study, the mapping data of Phoenix, Arizona is wrangled utilizing Python and queried using a local sqlite database.  The exact area of the map can be extracted at the following link:

https://mapzen.com/data/metro-extracts/metro/phoenix_arizona/

Note that the Raw OSM XML file was selected for download. 

## Challenges Encountered During Wrangling

Various challenges arised in the initial wrangling phase of the case study mostly relating to the lack of consistency across data entries.  Several of these inconsistencies are detailed below.


### Punctuation in Addresses

Street addresses in the dataset had variable amounts of punctuation depending on the entry. For example, one entry looks like the following:

    E. Marilyn Rd., Bldg. 4, Ste. 130

On the other hand, another street address entry in the same dataset looks as follows:

    West Larkspur Drive

To address this, all of the street name entries were debreviated to ensure consistency across the dataset.  The function update_name along with a couple of regular expressions accomplishes this. 


---------
```python 
#Regular expression to find street names and cardinal directions
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
direction_type_re = re.compile(r'(N |E |S |W |N\.|S\.|E\.|W\.)')


def update_name(name, mapping):
    '''this function updates the address name such that it unabbreviates street types, 
        directions, and removes commas in the addresses as to assure data consistency
        across all data entries'''
    m = street_type_re.search(name)
    n = direction_type_re.match(name)
    if n:
        name = re.sub(n.group(), mapping[n.group()], name)
        name = name.replace(',', '')
    if m.group() not in expected:
            if m.group() in mapping.keys():
                name = re.sub(m.group(), mapping[m.group()], name)
                name = name.replace(',' ,'')
    return name
 ```
 ----------

### Inconsistent Postal Code Formatting

Another issue in the data is the various representations of the postal codes.  The general formats are replicated below where 'X' represents a digit:

    (1) AZ XXXXX
    (2) XXXXX-XXXX
    (3) XXXXX

Since each format consisted of a sequence of five numbers that determine an entry's zip code, it seemed reasonable to condense each postal code entry to the one depicted by (3).  This was done by integrating the update_postcode function into the auditing process, which is replicated below.  

----
```python
def update_postcode(postcode):
    '''This function updates the postcodes to a standard 5 digit number'''
    if 'AZ' in postcode:
        new_post_code = postcode.strip('AZ ')[0:5]
    else:
        new_post_code = postcode[0:5]
    return new_post_code
```
----

The above function did not resolve all of the postal code formatting inconsistencies, however. To examine this further, a simple query to filter out any post code value above 85999 and less than 85000 (since all Phoenix post codes begin with '85') can be performed.  This query, along with its output is replicated below.  

```mysql
SELECT value, count(*) as num FROM node_tags
WHERE key = 'postcode'AND (value > 85999 OR value < 85000)
GROUP BY value ORDER BY num;
```

----
    Post Code Count 
    82381      1
    83003      1
    92127      1
    95295      1
    99352      1
    
----

It looks like there are at least five additional postal codes that need to either be corrected or removed from the database. Another query can be used to identify the id values for each of these post codes. Then, each respective entry can be investigated further to determine if the post code is a potential typo or just a misplaced data entry.   

```mysql
SELECT id, value FROM node_tags WHERE key = "postcode" 
AND (value = "82381" OR value = "83003" OR value = "92127" OR value = "95295" OR value = "99352");
```

----
         id      Post Code
     150964064     83003
    1383542330     82381
    4605850963     92127
    4658055481     95295
    4792771037     99352
    
----

```mysql
SELECT id, value FROM node_tags 
WHERE (key = 'state') AND 
(id = 4792771037 OR id = 150964064 OR id = 1383542330 OR id = 4605850963 OR id = 4658055481);
```

----
        id      State
    1383542330    AZ
    4605850963    CA
    4658055481    AZ
    4792771037    WA
    
----

From the above table, it looks like a couple of the entries are not even from Arizona, so they should be deleted accordingly from both the node and node_tags tables respectively.

```mysql
DELETE FROM node WHERE id = 4792771037;
DELETE FROM node_tags WHERE id = 4792771037;
DELETE FROM nodes WHERE id = 4605850963;
DELETE FROM node_tags WHERE id = 4605850963;
```

The additional two entries with postal code discrepancies and state data will need to be investigated further.  It would be useful to look into the actual addresses of each entry.

```mysql
SELECT id, value FROM node_tags 
WHERE (key = 'housenumber' OR key = 'street') 
AND (id = 1383542330 or id = 4658055481);
```

----
        id                Value
    1383542330  West Larkspur Drive
    1383542330                 8215
    4658055481                 1925
    4658055481  South Coronado Road
    
----

A quick google search of the following addresses, "8215 West Larkspur Drive" and "1925 South Coronado Road", shows that the actual zip codes of these addresses are "85381" and "85925" respectively. These can be updated with the following SQL queries:

```mysql 
UPDATE node_tags
SET value = '85381'
WHERE id = 1383542330 and key = 'postcode';
```

```mysql
UPDATE node_tags
SET value = '85295'
WHERE id = 4658055481 and key = 'postcode';
```

The final post code value that needs to be dealt with is the entry with corresponding id = '150964064'.  Looking into the key values for each of its entries, it looks like the information was collected from Wikipedia.  Wikipedia pointed to another website in order to get the postal code information and that website can be found using the below query:

```mysql
SELECT id, key, value FROM node_tags 
WHERE key = 'website' 
AND id = 150964064;
```

----
        id      key            value
    150964064  website  http://phoenix.gov/
    
----

Going to the website mentioned, and scrolling down to the bottom of page, it is clear that the postal code should have been "85003" instead of "83003".  A screenshot of this is shown below.


----





<img src="phx_screenshot.png">

----

Finally, the entry can be updated in the node_tags table so that the postal code accurately reflects the data entry. 

```mysql
UPDATE node_tags
SET value = '85003'
WHERE id = 150964064 and key = 'postcode';
```

## Overview of the Phoenix Area

After wrangling, auditing, and correcting the data, it can now be further explored utilizing a series of SQL queries. Some statistics will first be summarized such as the size of the datasets, the contributor information, and the number of nodes and ways. Then, certain nodes will be investigated deeper. 


### Size of the Data
----
        File             Size
    phoenix_arizona.osm  700 MB
    phoenix_az.db        404 MB
    nodes.csv            256 MB
    node_tags.csv         11 MB
    ways.csv              25 MB
    ways_tags.csv         87 MB
    ways_nodes.cv         67 MB  
    
----

### Number of Unique Users

```mysql
SELECT COUNT(DISTINCT(nodes_and_ways.uid))          
FROM (SELECT uid FROM node UNION ALL SELECT uid FROM way) nodes_and_ways;

```

----
1517

----

### Number of Nodes and Ways

```mysql
SELECT count(id) from node;
```

  ----
    3157146

----

```mysql
SELECT count(id) from way;
```

----
    427043

----
    

### Most Common Types of Nodes

```mysql
SELECT key, count(*) as num from node_tags
GROUP BY key 
ORDER BY num DESC
LIMIT 20;
```

  ----
       Node Key  Count
        highway  87711
          power  24577
        natural  20466
       crossing  16928
        bicycle  11231
          horse  11099
     supervised  11022
           name  10042
        amenity   7860
          state   7091
           city   6939
         street   6569
    housenumber   6478
     created_by   6334
        barrier   6047
       postcode   4585
        country   4230
        leisure   2914
         noexit   2676
 ----

It would be interesting to see the kind of culture Phoenix represents by examining the amenities available. 

### TOP AMENITIES

```mysql
SELECT value, count(*) as num from node_tags
where key = 'amenity'
GROUP BY value ORDER BY num DESC
LIMIT 15;
```

----
              Amenity  Count
               bench   1114
          restaurant    776
           fast_food    720
                fuel    685    
    place_of_worship    614
        waste_basket    609
              school    433
            fountain    250
                bank    219
            pharmacy    174
             toilets    172
                cafe    171
        swimming_pool   146
             parking     97
        trailer_park     91
    
----


To provide a deeper analysis of the Phoenix area, the  religious preferences, food choices, and shopping options provided to Phoenicians are detailed below.  

### Religious Ideology


To learn about the religious preferences of Phoenix denizens, the total number of places of worship for each faith can be queried by joining node_tags table to a temporary table 'A' conditioned on their id's being equivalent. 

```mysql
SELECT value, count(*) as num 
FROM node_tags
    JOIN (SELECT DISTINCT(id) FROM node_tags WHERE value = 'place_of_worship') A
    ON node_tags.id = A.id
WHERE key = 'religion' 
GROUP BY value ORDER BY num DESC
LIMIT 5;
```

----
    Religion         Total Places of worship
    christian                563
    jewish                    4
    bahai                     2
    buddhist                  1
    muslim                    1
    
----

Unsurprisingly, a preponderance of Phoenix places of worship are Christian, with a just a handful of other religious places of worship being represented.  

### Types of Cuisine

```mysql
SELECT value, count(*) as num 
FROM node_tags
    JOIN (SELECT DISTINCT(id) FROM node_tags WHERE value = 'restaurant') A
    ON node_tags.id = A.id
WHERE key = 'cuisine' 
GROUP BY value ORDER BY num DESC
LIMIT 10;
```

----
    Type of Cuisine  Count
             pizza     58
           mexican     57
          american     36
          sandwich     28
           italian     26
            burger     18
           chinese     16
             sushi     14
              thai     11
             asian     10
             
----

It looks like Phoenix residents enjoy their fair share of pizza and Mexican cuisine.  It would be interesting to compare these numbers to other major metropolitan areas based on the diversity of their residents.  Phoenix, which has the sixth-highest Hispanic share of its population, reflects this diversity in their cuisines available to them.

## Additional Suggestions to the Phoenix Dataset

As exemplified by the conundrum that was the inconsistency of the various postal codes within the Phoenix map data, it is clear that data quality is often compromised when human error is introduced.  An overwhelming majority of the postal code entries (all but 5) were easily dealt with a simple Python function; however, cleaning the rest of the five entries was a tedious process invoking several SQL queries.  

In order to fix this issue, it would be useful to have some error correcting mechanism such that if a user inputs a postal code that is not a part of the respective city, it flags an error and reports this error to a privileged OSM user.  Then, up to the superuser's discretion, she can choose to either modify the entry, notify the user of his incorrect entry, or leave as is if there is actually no error.  This would require additional maintenance for data assurance.  Additional problems are illustrated in the following queries outlined below. 



### Most Popular Restaurants

```mysql
SELECT value, count(*) as num 
FROM node_tags
    JOIN (SELECT DISTINCT(id) FROM node_tags WHERE value = 'restaurant') A
    ON node_tags.id = A.id
WHERE key = 'name' 
GROUP BY value ORDER BY num DESC
LIMIT 10;
```

---
                Restaurant  Count
                   Denny's     16
         Peter Piper Pizza      8
                 Pizza Hut      8
             Barro's Pizza      6
         Native New Yorker      6
       Streets of New York      6
                  Chipotle      5
    Chipotle Mexican Grill      5
                    Subway      5
        Buffalo Wild Wings      4
---

At first glance,  the top two restaurants look to be Denny's and Peter Piper Pizza.  However, Chipotle and Chipotle Mexican Grill are clearly the same restaurant and if that is taken into account, it becomes the second most popular restaurant.  A similar issue is demonstrated by cell phone providers for Phoenix. 

### Types of Shops in Phoenix

```mysql
SELECT value, count(*) as num FROM node_tags
WHERE key = 'shop'
GROUP BY value ORDER BY num DESC
LIMIT 6;
```

----
       Shop Type   count
     convenience    562
     supermarket    188
         clothes    161
      car_repair    110
          beauty     75
    mobile_phone     74
    
----

### Most Popular Cell Phone Providers

```mysql
SELECT value, count(*) as num 
FROM node_tags
    JOIN (SELECT DISTINCT(id) FROM node_tags WHERE value = 'mobile_phone') A
    ON node_tags.id = A.id
WHERE key = 'name' 
GROUP BY value ORDER BY num DESC
LIMIT 8;
```

----
            Provider  count
            T-Mobile     16
    Verizon Wireless     11
             Cricket     10
                AT&T      7
              Sprint      7
        Boost Mobile      6
            MetroPCS      5
           Metro PCS      2
       
----

Clearly, MetroPCS and Metro PCS represent the same provider yet are depicted as distinct.  This issue can be resolved by data harmonization efforts.  This could include a mapping function that renames several possible cell phone provider pseudonyms to a "golden standard" nomenclature list as determined by a group of OSM superusers. This solution could be extended to other nodes such as restaurants.  Of course, this method could run into issues when new restaurants or providers open in the area and are not yet listed.  Furthermore, it still does not account for user typos and other entry errors, which would require additional maintenance if implemented.  

By implementing the above fixes for the incorrect zip codes and naming discrepancies among the various providers/restaurants, both benefits and issues will be realized and can be summarized as follows:

**Benefits:**
* Less data cleaning performed at the user end 
* Increased data harmonization results in a more accurate dataset

**Anticipated Problems:**
* Increased code maintenance and efforts by OSM superuser
* New restaurants will have to be consistently updated in "golden standard" list resulting in additional maintenance

## Conclusion

The Open Street Map data for Phoenix is in general robust and varied, with over a thousand individuals contributing to over three million nodes and ways.  With so many contributors, errors in data quality are unavoidable, leading to necessary auditing if the data is to be used in any sort of productive fashion.  Attempts to audit were done so by cleaning up street names and postal codes; however, with the massive amounts of data encompassed within the dataset, additional cleaning steps could certainly be taken to ensure an even more functional dataset.  